In [1]:
import sys
import numpy as np
import pandas as pd
path_name = r"C:\Users\M543015\Desktop\OOMS\simulation"
sys.path.append(path_name+"/code")
#import natural_progression
#import treatment
#import create_cohort
#import library
ideal_flag = True
#if ideal_flag:
#    import ideal_screening as scr
#else:
#    import screening_distribution as scr
    
pd.set_option('display.max_columns', None)

In [2]:
###################################### CREATE COHORT ##############################################

In [3]:
#create dataframe that has a line for each person for each year of their life from age 1-100 for N IDs  with semi random gender
size = 100000
birth_year = 1960
ages = np.tile(np.arange(100), size) # from 1 to 100
IDs = np.repeat(np.arange(size), 100)
genders = np.repeat(np.round(np.random.rand(size)).astype(int),100)
df = pd.concat([pd.Series(ages, name = "age"), 
                     pd.Series(IDs, name = "ID"),
                     pd.Series(genders, name = "gender")], axis = 1)



In [4]:
#df.to_csv("1 Initial_cohort.csv")

In [5]:
#Get life expectancy 

"""
Life expectancy distribution obtained from the cohort life table from SSA - 
https://www.ssa.gov/oact/NOTES/as120/LifeTables_Body.html
In these tables, we get the survival function for males and females born
in given decade. I'm just using 1960 right now, in theory you could use another
decade if I downloaded all the files for each decade born.
Using the survival function, get the CDF, then use that to draw from the
underlying probability distribution.
"""



def draw_using_cdf(y,cdf,N):
    """ Given a discretized CDF, draw N points
    from the distribution. This should work for any N, including N = 1.
    """
    interval = y[1]-y[0]
    left_edge = y[0:len(y)-1]

    values = np.random.rand(N)
    value_bins = np.searchsorted(cdf, values)
    random_from_cdf = left_edge[value_bins]+np.random.rand(N)*interval
    return(random_from_cdf)


filename =  path_name+'/supplementary_files/'+'cohort_life_table_'+str(1960)+'_birth_year.csv'
life_table = pd.read_csv(filename)
life_table.dropna(inplace=True)
life_table.lxFemale = life_table.lxFemale.str.replace(',','').astype(float)/1e5
life_table.lxMale = life_table.lxMale.str.replace(',','').astype(float)/1e5
life_table['cdfFemale'] = 1.0-life_table.lxFemale
life_table['cdfMale'] = 1.0-life_table.lxMale

female_index =  df[ df.gender == 1].ID.unique()
male_index =  df[ df.gender == 0].ID.unique()
n_female = len(female_index)
n_male =  size - n_female

female_deaths = draw_using_cdf(life_table.xFemale.values,life_table.cdfFemale.values,n_female)
male_deaths = draw_using_cdf(life_table.xMale.values,life_table.cdfMale.values,n_male)


df.loc[ df.gender == 1, "death"] = np.repeat(female_deaths,100)
df.loc[ df.gender == 0, "death"] = np.repeat(male_deaths,100)
#df.to_csv("2 Life_expectancy.csv")

In [6]:
#Remove people who die before age 50
"""
Self explanatory. If the patient dies before screening can be even initiated,
the screening cannot affect anything. We want to study the group of patients
whom screening may affect. 
"""

df =  df[ df.death>=50].copy()
df.reset_index(inplace=True, drop = True)
size = len( df.ID.unique())

In [7]:
#df.to_csv("3 Life_expectancy_minus_dead_before_50.csv")

In [8]:
#Get cohort size to spot check 
cohort_size = []
cohort_size += [len(df.ID.unique())]
cohort_size

[91257]

In [9]:
###################################### Progression ##############################################


#get the natural progression of the cancer - without intervention

In [10]:
def generate_log_normal(m,v, N):
    """
    This generates draws from a log normal distribution, i.e. X~ log normal. The value 
    m is the mean of x observed, while v is a parameter we should tune. Then we calculate 
    the mean and variance of the Gaussian distribution for y = ln(x), create draws for y,
    and return x = e^y.
    """
    mean = np.log(m/np.sqrt(1.0+v/m**2.0))
    variance = np.log(1.0+v/m**2.0)
    time = np.random.randn(N)*np.sqrt(variance)+mean
    return(np.exp(time))
    
def create_correlated_indices(N, amount):
    new = np.round(np.arange(N)+np.random.randn(N)*amount).astype(int)
    new[new<0] = np.abs(new[new<0])
    new[new>N-1] = (N-1)-np.abs((N-1)-new[new>N-1])
   # new[new<0] = np.abs(np.random.randn((new<0).sum())*amount)
   # new[new>N-1] = N-np.abs(np.random.randn((new>N-1).sum())*amount)
    return(new)

In [11]:

#Make a copy of the cohort 

adenomadf = df.copy()

In [12]:
# Instantiate adenoma
#assigns adenoma (T/F) and location (rectum/colon) as a function of age, counts, and semi randomness
"""
Source: Appendix 1, Table 1, J Natl Cancer Inst 2009;101:1412–1422
The prevalence of adenomas is treated *only* as a function of age here.
The significance of colon vs rectal adenomas is not treated yet.
"""
ages = adenomadf.age.values

counts = np.array([0.4,1.3, 2.9, 5.8,13.0,28.2,51.9,88.3,136.7,202.4,279.0,351.7,417.6,442.9])/1e5
values = np.zeros(100)
edges = np.arange(len(counts))*5+20
for i in range(len(edges)-1):
    values[edges[i]:edges[i+1]] = counts[i]
values[edges[i+1]:] = counts[i+1]
adenoma = np.where(ages<=99,np.random.rand(len(ages))<values[ages], False )
loc = np.random.rand(len(ages)) 
location = np.where(loc<0.31, "rectum", "colon")
adenomadf["adenoma"] = adenoma
adenomadf["location"] = location
adenomadf = adenomadf.loc[adenomadf.adenoma].copy() #Filters to only those who adenoma ==T



In [13]:
#adenomadf.to_csv("4 Adenoma_assigned_by_age.csv")

In [14]:
#Only keep people who are younger than their death year
#Aka remove people who have already died 

adenomadf = adenomadf[adenomadf.age<adenomadf.death].copy()

In [15]:
#adenomadf.to_csv("5 Adenoma_minus_already_dead.csv")

In [16]:
#Determine if cancerous 
#adds progression column (T/F) based on age fuction

"""
Source: Appendix 1, Table 1, J Natl Cancer Inst 2009;101:1412–1422
Probability adenoma progresses
0–45 years: linearly increasing from 0 to 3%
45–65 years: linearly increasing from 3% to 17%
65–100 years: linearly decreasing 17% to 13%
"""
age = adenomadf.age.values
probs = np.where(age<45, 0.03*age/45, np.where(age>65, -0.04/35*age+0.244286, 0.14/20*age-0.285))
adenomas_progressive = np.random.rand(len(age))<probs
adenomadf["progressive"] = adenomas_progressive


In [17]:
#adenomadf.to_csv("6 Determine_if_cancerous.csv")

In [18]:
#Generate Timescales

"""
Given mean times as an adenoma, in the various stages, and in the preclinical
cancerous state, generate times for the duration in each state from log-normal
distributions. The second param should be tuned/calibrated against some data set.
Determine stage and age at diagnosis.
"""

N = adenomadf.progressive.sum()
N_progressive = N# some of these may go undetected!
adenoma_time = 16.4
stage1 = 2
stage2 = 1
stage3 = 1.5
stage4 = 0.8
preclinical = 3.6
indices = adenomadf.progressive == True
noninvasive_adenoma_times = np.random.exponential(scale = adenoma_time, size = N)
stage1_times = np.random.exponential(scale = stage1, size = N)
stage2_times = np.random.exponential(scale = stage2, size = N)
stage3_times = np.random.exponential(scale = stage3, size = N)
stage4_times = np.random.exponential(scale = stage4, size = N)

#Create a correlation between adenoma time and the time in each stage
#        indices1 = create_correlated_indices(N, int(N/10))
#        indices2 = create_correlated_indices(N, int(N/10))
#        indices3 = create_correlated_indices(N, int(N/10))
#        indices4 = create_correlated_indices(N, int(N/10))
adenomadf.loc[indices,"AGT"] = noninvasive_adenoma_times
adenomadf.loc[indices,"SI"] = stage1_times#[indices1]
adenomadf.loc[indices,"SII"] =  stage2_times#[indices2]
adenomadf.loc[indices,"SIII"] =  stage3_times#[indices3]
adenomadf.loc[indices,"SIV"] =  stage4_times#[indices4]
adenomadf.loc[indices,"PCT"] = np.random.exponential(scale = preclinical, size = N)
adenomadf["dxed_no_screening"] = "null"
adenomadf.loc[adenomadf.SI+adenomadf.SII+adenomadf.SIII+adenomadf.SIV<adenomadf.PCT,"dxed_no_screening"] = "SIV"
adenomadf.loc[adenomadf.SI+adenomadf.SII+adenomadf.SIII+adenomadf.SIV>adenomadf.PCT,"dxed_no_screening"] = "SIV"
adenomadf.loc[adenomadf.SI+adenomadf.SII+adenomadf.SIII>adenomadf.PCT,"dxed_no_screening"] = "SIII"
adenomadf.loc[adenomadf.SI+adenomadf.SII>adenomadf.PCT,"dxed_no_screening"] = "SII"
adenomadf.loc[adenomadf.SI>adenomadf.PCT,"dxed_no_screening"] = "SI"
adenomadf["age_at_dx_no_screening"] = adenomadf.PCT+adenomadf.AGT+adenomadf.age

    # Keep only examples that are progressive
    # Question: should we further restrict to those who would not get a dx before death? This 
    #would skew the adenoma growth time and pre clinical time to shorter values if we did.
    # but if never dx-ed, their life isnt affected by it in terms of life years.
    #the means are 7 (!) and 2.6. I do this later anyways...
    


In [19]:
#noninvasive_adenoma_times

In [20]:
#adenomadf.to_csv("7 Generate_timelines.csv")

In [21]:
#Clean up - keep only progressive ... pointless as they've already been filtered out previously

adenomadf = adenomadf[((adenomadf.progressive == True))].copy()


In [22]:
###################################### Screening ######################################################


#now get the screening - only for pats with a progressive adenoma to speed it up


#Filter the original cohort by the IDs of people who are progressive as found above
foo1 = adenomadf[["ID","age"]].merge(df, how = "left", on = "ID", suffixes = ('_tmp',''))
foo1.drop("age_tmp", inplace=True, axis = 1)

#Output is a df with only the IDs that are progressive with one line per each year of their life 0-99
#and their gender and that is it 

In [23]:
#################################### Ideal screening  #########################
# makes a copy of the empty cohort and adds an empty number type column with Not A Numbers 
screeningdf = df
N = len(df.ID.unique())
screeningdf["type"] = np.nan



In [24]:
# Picks a type of test to give them at age 50 by weighted semi randomness 


def get_type_of_test(N):
    """ Return N test types. I'm using probabilities based off the the number of 
    each procedure that we have in KP data for this population, restricted to
    first and second tests, and only tests prior to diagnoses. >2008.
    type       P(type)
    colon      0.12
    fit        0.87
    sigmoid    0.01
    """
    type_random = np.random.rand(N)
    test_type = np.array(["fit"]*N,  dtype='<U7')
    test_type[type_random>0.99] = "sigmoid"
    test_type[type_random<0.12] = "colon"       
    return(test_type)

types = get_type_of_test(N)

#assuming age of first test at 50
times = 50+np.zeros(N)
IDS = screeningdf["ID"].unique()     
newdf = pd.DataFrame(data = {"ID":IDS,"age":times, "type":types})
screeningdf = screeningdf.merge(newdf, how = "left", on = ["ID","age"])
screeningdf.loc[screeningdf.type_x.isnull() & ~screeningdf.type_y.isnull(),"type_x"] = \
screeningdf.loc[screeningdf.type_x.isnull() & ~screeningdf.type_y.isnull(),"type_y"]
screeningdf.rename(columns = {"type_x":"type"}, inplace=True)
screeningdf.drop("type_y", axis =1 , inplace=True)

In [25]:
#spot check 
#screeningdf[screeningdf['type'].isnull() ==False].head()

#screeningdf.to_csv("8 First_test_at_50.csv")

In [26]:
def get_type_of_test_given_previous(last_type):
    """ Return N test types. I'm using probabilities based off the the number of 
    each procedure that we have in KP data for this population, restricted to
    first and second tests, and only tests prior to diagnoses. >2008.  
    first_type second_type    P(second type)            
    colon      colon          0.64
               fit            0.34
               sigmoid        0.02
    fit        colon          0.08
               fit            0.91
               sigmoid        0.01
    sigmoid    colon          0.21
               fit            0.76
               sigmoid        0.03
    """
    N = len(last_type)
    test_type = np.array(["fit"]*N,  dtype='<U7')
    type_random = np.random.rand(N)
    test_type[((last_type=="colon") & (type_random<0.64))] = "colon"
    test_type[((last_type=="colon") & (type_random>0.98))] = "sigmoid"
    test_type[((last_type=="sigmoid") & (type_random<0.21))] = "colon"
    test_type[((last_type=="sigmoid") & (type_random>0.97))] = "sigmoid"    
    test_type[((last_type=="fit") & (type_random<0.08))] = "colon"
    test_type[((last_type=="fit") & (type_random>0.99))] = "sigmoid"
    return(test_type)

#This fills in all the tests they would get over the rest of their lives


for i in range(25):#I guess in theory a person could have a fit test each year 50-75
        last_test_index = screeningdf[~screeningdf.type.isnull()][['ID',
                                           'age']].groupby(['ID']).idxmax().values.flatten()
        last_test_df = screeningdf.loc[last_test_index].copy()
        last_test_type = last_test_df.type.values
        #types = get_type_of_test(N)
        types = get_type_of_test_given_previous(last_test_type)
        previous_age = last_test_df.age
        last_test_df["delta"] = 0
        last_test_df.loc[last_test_df.type=="fit","delta"] = 1
        last_test_df.loc[last_test_df.type=="colon","delta"] = 10
        last_test_df.loc[last_test_df.type=="sigmoid","delta"] = 5
        ages = (previous_age+last_test_df.delta).values
        IDS = last_test_df.ID.values
        newdf = pd.DataFrame(data = {"ID":IDS,"age":ages, "type":types})

        newdf[["ID","age"]] = newdf[["ID","age"]].astype(int)
        screeningdf = screeningdf.merge(newdf, how = "left", on = ["ID","age"])
        screeningdf.loc[screeningdf.type_x.isnull() & ~screeningdf.type_y.isnull(),"type_x"] = \
        screeningdf.loc[screeningdf.type_x.isnull() & ~screeningdf.type_y.isnull(),"type_y"]
        screeningdf.rename(columns = {"type_x":"type"}, inplace=True)
        screeningdf.drop("type_y", axis =1 , inplace=True)

In [27]:
#spotcheck

#screeningdf[(screeningdf['ID']==135) & (screeningdf['age']>50)]

#screeningdf.to_csv("9 All_screenings_50_70.csv")

In [28]:
def get_up_to_date_years(df):
    """
    Columns expected to be ID, age, and type of test recieved at that age
    """
    df = df[((df.age>=50) & (df.age<=75))].copy()
    ages = pd.DataFrame({"age":np.arange(26)+50})
    IDs = pd.DataFrame({"ID":df.ID.unique()})
    ages['key']=0
    IDs['key']=0
    pat_df = ages.merge(IDs, how = "outer", on = 'key')
    pat_df.drop('key',axis=1,inplace=True)
    pat_df.sort_values(["ID", "age"],inplace = True)
    pat_df.reset_index(inplace = True, drop = True)
    
    df = pat_df.merge(df, how = "left", on = ["ID", "age"])
    df["up_to_date"] = False
    df.loc[df.type == "fit","up_to_date"] = True
    for i in np.arange(10):
        df['up_to_date'] = (((df.shift(i).ID == df.ID) & (df.shift(i).type=='colon'))\
          | df.up_to_date == True)
    for i in np.arange(5):
        df['up_to_date'] = (((df.shift(i).ID == df.ID) & (df.shift(i).type=='sigmoid'))\
          | df.up_to_date == True)
    return(df)

utd = get_up_to_date_years(screeningdf[["ID","age","type"]].copy())
#utd.head()

In [29]:
up_to_date_rate = []

up_to_date_rate += [utd.up_to_date.sum()/len(utd)]
up_to_date_rate #spot checks to make sure this worked should be 1 aka 100%

[1.0]

In [30]:
utd.up_to_date.value_counts(dropna=False) 

True    2372682
Name: up_to_date, dtype: int64

In [31]:
screeningdf = screeningdf[~screeningdf["type"].isnull()].copy()
screeningdf = screeningdf[screeningdf.age<76].copy()
merged = adenomadf.merge(screeningdf[["ID","age","type"]], 
                  how = "left", on = "ID", suffixes = ('','_test'))
##### Get stage test would catch it at if it were a positive
merged["years_to_test_from_adenoma"] = merged.age_test-merged.age
merged["possible_interruption"] = merged.years_to_test_from_adenoma>=0
merged["screening_dxed"] = "None"
merged.loc[((merged.possible_interruption) & 
                      (merged.AGT+merged.SI+
                       merged.SII+merged.SIII+
                       merged.SIV<
                       merged.years_to_test_from_adenoma)),"screening_dxed"] = "SIV"
merged.loc[((merged.possible_interruption) & 
                      (merged.AGT+merged.SI+
                       merged.SII+merged.SIII+
                       merged.SIV>
                       merged.years_to_test_from_adenoma)),"screening_dxed"] = "SIV"
merged.loc[((merged.possible_interruption) & 
                      (merged.AGT+merged.SI+
                       merged.SII+merged.SIII>
                       merged.years_to_test_from_adenoma)),"screening_dxed"] = "SIII"
merged.loc[((merged.possible_interruption) & 
                      (merged.AGT+merged.SI+
                       merged.SII>
                       merged.years_to_test_from_adenoma)),"screening_dxed"] = "SII"
merged.loc[((merged.possible_interruption) & 
                      (merged.AGT+merged.SI>
                       merged.years_to_test_from_adenoma)),"screening_dxed"] = "SI"
merged.loc[((merged.possible_interruption) & 
                      (merged.AGT>
                       merged.years_to_test_from_adenoma)),"screening_dxed"] = "Adenoma"
##### Sensitivity numbers = 1.0-FNR
FIT_adenoma = 0.12
FIT_cancer = 0.7
sigmoid_colon_adenoma = 0.85
sigmoid_colon_cancer= 0.95
merged["false_negative_prob"] = 0.0
merged.loc[((merged.type=="fit") & 
            (merged.screening_dxed=="Adenoma")), "false_negative_prob"] = 1.0-FIT_adenoma
merged.loc[((merged.type=="fit") & 
            ((merged.screening_dxed!="Adenoma") &
             (merged.screening_dxed!="None"))), "false_negative_prob"] = 1.0-FIT_cancer
merged.loc[((merged.type=="colon") & 
            (merged.screening_dxed=="Adenoma")), "false_negative_prob"] = 1.0-sigmoid_colon_adenoma
merged.loc[((merged.type=="colon") & 
            ((merged.screening_dxed!="Adenoma") &
             (merged.screening_dxed!="None"))), "false_negative_prob"] = 1.0-sigmoid_colon_cancer
merged.loc[((merged.type=="sigmoid") & 
            (merged.screening_dxed=="Adenoma")), "false_negative_prob"] = 1.0-sigmoid_colon_adenoma
merged.loc[((merged.type=="sigmoid") & 
            ((merged.screening_dxed!="Adenoma") &
             (merged.screening_dxed!="None"))), "false_negative_prob"] = 1.0-sigmoid_colon_cancer
merged["test_result"] = False
possible_interruptions = (merged.possible_interruption==True).sum()
chance = np.random.rand(possible_interruptions)
merged.loc[merged.possible_interruption==True,"test_result"] = \
chance >= merged.loc[merged.possible_interruption==True,"false_negative_prob"]
merged = merged[merged.test_result == True].copy()
first_age_interrupted = merged[["ID","age","age_test"]].groupby(["ID","age"]).idxmin()
merged = merged.loc[first_age_interrupted.values.flatten()].copy()
col1 = adenomadf.columns
col2 = merged.columns
join_columns = list(set(col1).intersection(set(col2)))
adenomadf = merged.merge(adenomadf,how = "outer", on = join_columns)
final_results = adenomadf
#final_results.to_csv("10 Determie_first_positive.csv")







In [32]:
#with no screening:
final_results["final_dx"] = final_results.dxed_no_screening
final_results["age_at_final_dx"] = final_results.age_at_dx_no_screening
#Replace if screening led to a positive test result
indices = final_results.test_result==True
final_results.loc[indices,"final_dx"] = final_results.loc[indices,"screening_dxed"]
final_results.loc[indices,"age_at_final_dx"] = final_results.loc[indices,"age_test"]

In [33]:
##################################  Gets summary data   #########################################


#Some people develop cancer so late in life that we would not diagnosis them with it
#unless we had screening implemented. This counterintuitively means we might know of more
#cancer diagnoses with screening than without. But the underlying numbers are different. 
# Screening also reduces the number of cancer diagnoses by preventing cancer, though.
#People with cancer before death, people diagnosed with cancer, people diagnoses with adenoma.

#Underlying:
n_pats_w_prog_adenomas = len(final_results.ID.unique())
final_results["first_cancerous_time"] = final_results.age+final_results.AGT
n_cancerous_before_death_no_screening = len(final_results[final_results.first_cancerous_time<=final_results.death].ID.unique())
n_dxed_no_screening = len(final_results[final_results["age_at_dx_no_screening"]<=final_results.death].ID.unique())
n_adenoma_no_screening = len(final_results[final_results.death-final_results.age<=final_results.AGT].ID.unique())
cancerous_pats = final_results[final_results.first_cancerous_time<final_results.death].copy()
prevented = len(cancerous_pats[cancerous_pats.final_dx=="Adenoma"].ID.unique())
#Detected:
final_results = final_results[final_results.age_at_final_dx<final_results.death].copy()
adenomas_removed = len(final_results[final_results.final_dx=="Adenoma"].ID.unique())# this only happens with screening
n_pats_dxed_with_cancer = len(final_results[final_results.final_dx!="Adenoma"].ID.unique())


In [34]:
############################ calculate for affected group ############################

tmp = final_results[final_results["age_at_dx_no_screening"]<=final_results.death].copy()
adenomas_removed = len(tmp[tmp.final_dx=="Adenoma"].ID.unique())# this only happens with screening
n_pats_dxed_with_cancer = len(tmp[tmp.final_dx!="Adenoma"].ID.unique())


In [35]:
#################### people who didn't get screened but got dx before death ############################

df1 = final_results[final_results.age_at_dx_no_screening<final_results.death].copy()
df1.head()

age    ID  gender       death adenoma location progressive       AGT  \
1   63   326       1   91.289459    True    colon        True  5.981044   
3   73   878       1  100.747850    True    colon        True  1.410019   
4   66  1793       0   81.630761    True   rectum        True  7.664940   
5   70  1793       0   81.630761    True   rectum        True  5.065323   
7   65  2034       0   89.649222    True    colon        True  8.867504   

         SI       SII      SIII       SIV       PCT dxed_no_screening  \
1  0.396665  0.189388  2.791951  0.280256  4.062896               SIV   
3  2.223541  0.425170  0.186668  1.327467  3.854825               SIV   
4  1.341764  0.237118  1.024294  0.709252  2.947965               SIV   
5  0.671101  0.001675  1.436631  1.078467  4.213508               SIV   
7  1.272514  0.447215  1.781689  0.474361  4.915267               SIV   

   age_at_dx_no_screening  age_test   type  years_to_test_from_adenoma  \
1               73.043940      69.0    fit                         6.0   
3               78.264844      75.0  colon                         2.0   
4               76.612905      74.0  colon                         8.0   
5               79.278831      74.0  colon                         4.0   
7               78.782770      66.0  colon                         1.0   

  possible_interruption screening_dxed  false_negative_prob test_result  \
1                  True             SI                 0.30        True   
3                  True             SI                 0.05        True   
4                  True             SI                 0.05        True   
5                  True        Adenoma                 0.15        True   
7                  True        Adenoma                 0.15        True   

  final_dx  age_at_final_dx  first_cancerous_time  
1       SI             69.0             68.981044  
3       SI             75.0             74.410019  
4       SI             74.0             73.664940  
5  Adenoma             74.0             75.065323  
7  Adenoma             66.0             73.867504

In [36]:
def exponential_survival_time(df, diagnosis_column):
    """
    Source: http://www.beseengetscreened.com/blog/colon-cancer-stages.php
    If the time to death after diagnosis distribution is an exponential, knowing
    the 5 year survival probability is enough to know the distribution.
    If P(x) = lambda*np.exp(-lambda *x), and the chance of survival past
    5 years is y, then lambda = -np.log(y)/5.0. Python takes the "scale" = 1/lambda.
    SI: 0.94
    SII: 0.82
    SIII: 0.67
    SIV: 0.11
    """
    #Adenomas do not lead to death directly. Set a large survival time for all,
    #then replace the stage entries with smaller numbers.
    df["survival_time"] = np.zeros(len(df))+100
    lam_SI = -np.log(0.94)/5.0
    lam_SII = -np.log(0.82)/5.0
    lam_SIII = -np.log(0.67)/5.0
    lam_SIV = -np.log(0.11)/5.0
    size_SI = (df[diagnosis_column]=="SI").sum()
    size_SII = (df[diagnosis_column]=="SII").sum()
    size_SIII = (df[diagnosis_column]=="SIII").sum()
    size_SIV = (df[diagnosis_column]=="SIV").sum()

    df.loc[(df[diagnosis_column]=="SI"),"survival_time"] = np.random.exponential(scale = 1.0/lam_SI, size = size_SI)
    df.loc[(df[diagnosis_column]=="SII"),"survival_time"] = np.random.exponential(scale = 1.0/lam_SII, size = size_SII)
    df.loc[(df[diagnosis_column]=="SIII"),"survival_time"] = np.random.exponential(scale = 1.0/lam_SIII, size = size_SIII)
    df.loc[(df[diagnosis_column]=="SIV"),"survival_time"] = np.random.exponential(scale = 1.0/lam_SIV, size = size_SIV)

    return(df)

In [37]:
#################### people who didn't get screened Surival Time ############################

#creates a survival time based on the stage and weighted semi randomness

df1 = exponential_survival_time(df1,'dxed_no_screening')

In [38]:
#df1.to_csv("11 Not Screened Survival Post Diagnosis.csv")
#df1.head()

In [39]:



def survival_time(df, diagnosis_column):
    """
    Source: http://www.beseengetscreened.com/blog/colon-cancer-stages.php
    KP data for # of years until mortality given that the patient does not survive for
    5 years after diagnosis, as a function of stage.
    
    Given a diagnosis in a given stage, how long will the patient survive? Either the patient
    dies within 5 years, and they get a specific number, or else they live longer than 
    five years and no specific number is returned. KP data here is incomplete: we do not have a long 
    enough observation window on our patients to know how long they survive after a diagnosis
    past the window of five years. That is, we have censored data. More importantly, though, we do
    not know the cause of death.
    """
    df["survival_prob_given_stage"] = np.random.rand(len(df))
    df["survival_time"] = np.random.rand(len(df))
    df.loc[df[diagnosis_column] == "Adenoma", "five_year_survival"] = True
    df.loc[df[diagnosis_column]=="SI","five_year_survival"] = \
    df.loc[df[diagnosis_column]=="SI","survival_prob_given_stage"] < 0.94
    df.loc[df[diagnosis_column]=="SII","five_year_survival"] = \
    df.loc[df[diagnosis_column]=="SII","survival_prob_given_stage"] < 0.82
    df.loc[df[diagnosis_column]=="SIII","five_year_survival"] = \
    df.loc[df[diagnosis_column]=="SIII","survival_prob_given_stage"] < 0.67
    df.loc[df[diagnosis_column]=="SIV","five_year_survival"] = \
    df.loc[df[diagnosis_column]=="SIV","survival_prob_given_stage"] < 0.11
    
    #if you survive > 5 years, we assume you will not die from cancer, so just
    # set a large survival time.
    df.loc[df.five_year_survival == True, "survival_time"] = 100
    df.loc[((df[diagnosis_column]=="SI") & (df.five_year_survival==False)),"survival_time"] = \
    df.loc[((df[diagnosis_column]=="SI") & (df.five_year_survival==False)),"survival_time"]*0.2+2.1
    df.loc[((df[diagnosis_column]=="SII") & (df.five_year_survival==False)),"survival_time"] = \
    df.loc[((df[diagnosis_column]=="SII") & (df.five_year_survival==False)),"survival_time"]*0.2+2.0
    df.loc[((df[diagnosis_column]=="SIII") & (df.five_year_survival==False)),"survival_time"] = \
    df.loc[((df[diagnosis_column]=="SIII") & (df.five_year_survival==False)),"survival_time"]*0.2+1.8
    df.loc[((df[diagnosis_column]=="SIV") & (df.five_year_survival==False)),"survival_time"] = \
    df.loc[((df[diagnosis_column]=="SIV") & (df.five_year_survival==False)),"survival_time"]*0.2+1.3
    return(df)


def determine_years_lost(df, diagnosis_column, age_at_dx_column):
    df["actual_death"] = np.minimum(df[age_at_dx_column] + df.survival_time, df.death)
    ## A patient can have more than one bout with CRC. Take the one that would lead to the
    ## earliest death date. Note that this assumes the screening pattern would not
    ## change given an early adenoma or CRC dx, which is probably wrong (the pat would
    ## become high risk and be screened more)
    indices = df[["ID","actual_death"]].groupby("ID").idxmin().values.flatten()
    df = df.loc[indices].copy()
    years_lost = (df.death-df.actual_death).sum()
    mean_age_dx = df[df[diagnosis_column]!="Adenoma"][age_at_dx_column].mean()
    median_age_dx = df[df[diagnosis_column]!="Adenoma"][age_at_dx_column].median()


    index = df[df[diagnosis_column]!="Adenoma"][diagnosis_column].value_counts().index.values
    values = df[df[diagnosis_column]!="Adenoma"][diagnosis_column].value_counts().values
    s_counts =[[values[index=="SI"][0],values[index=="SII"][0], \
                 values[index=="SIII"][0], values[index=="SIV"][0]]]
    
    n_deaths = (df.actual_death!=df.death).sum()
    return(years_lost, mean_age_dx, median_age_dx, s_counts, n_deaths)



In [40]:
#################### people who didn't get screened Actual Death  ############################
yl, mean_age, median_age, \
    stage_counts, n_deaths = determine_years_lost(df1,'dxed_no_screening','age_at_dx_no_screening') 

In [41]:


df1.to_csv("12 Not Screened Actual Death.csv")

In [42]:
#################### people who got screened before they died  ############################

df = final_results[final_results.age_at_final_dx<final_results.death].copy()
df = exponential_survival_time(df,'final_dx')
#df.to_csv("13 Screened Survival Post Diagnosis.csv")


In [43]:
 yl, mean_age, median_age, \
    stage_counts, n_deaths = determine_years_lost(df,'final_dx','age_at_final_dx') 
#df.to_csv("14 Screened Actual Death.csv")

In [44]:


df1.columns

Index(['age', 'ID', 'gender', 'death', 'adenoma', 'location', 'progressive',
       'AGT', 'SI', 'SII', 'SIII', 'SIV', 'PCT', 'dxed_no_screening',
       'age_at_dx_no_screening', 'age_test', 'type',
       'years_to_test_from_adenoma', 'possible_interruption', 'screening_dxed',
       'false_negative_prob', 'test_result', 'final_dx', 'age_at_final_dx',
       'first_cancerous_time', 'survival_time', 'actual_death'],
      dtype='object')

In [45]:
df.head()

age    ID  gender       death adenoma location progressive        AGT  \
0   61   305       0   76.815018    True   rectum        True  19.848218   
1   63   326       1   91.289459    True    colon        True   5.981044   
2   74   831       0   83.528330    True   rectum        True  19.888000   
3   73   878       1  100.747850    True    colon        True   1.410019   
4   66  1793       0   81.630761    True   rectum        True   7.664940   

         SI       SII      SIII       SIV       PCT dxed_no_screening  \
0  1.295358  3.685916  1.024013  1.292070  3.571319               SII   
1  0.396665  0.189388  2.791951  0.280256  4.062896               SIV   
2  0.124096  1.623470  1.552733  0.405972  0.193731               SII   
3  2.223541  0.425170  0.186668  1.327467  3.854825               SIV   
4  1.341764  0.237118  1.024294  0.709252  2.947965               SIV   

   age_at_dx_no_screening  age_test   type  years_to_test_from_adenoma  \
0               84.419537      69.0  colon                         8.0   
1               73.043940      69.0    fit                         6.0   
2               94.081731      75.0  colon                         1.0   
3               78.264844      75.0  colon                         2.0   
4               76.612905      74.0  colon                         8.0   

  possible_interruption screening_dxed  false_negative_prob test_result  \
0                  True        Adenoma                 0.15        True   
1                  True             SI                 0.30        True   
2                  True        Adenoma                 0.15        True   
3                  True             SI                 0.05        True   
4                  True             SI                 0.05        True   

  final_dx  age_at_final_dx  first_cancerous_time  survival_time  actual_death  
0  Adenoma             69.0             80.848218     100.000000     76.815018  
1       SI             69.0             68.981044     147.404845     91.289459  
2  Adenoma             75.0             93.888000     100.000000     83.528330  
3       SI             75.0             74.410019     221.181999    100.747850  
4       SI             74.0             73.664940     101.625314     81.630761

In [46]:
sidebyside =pd.merge(df1, df, on=['age', 'ID', 'gender', 'death', 'adenoma', 'location', 'progressive',
       'AGT', 'SI', 'SII', 'SIII', 'SIV', 'PCT', 'dxed_no_screening',
       'age_at_dx_no_screening', 'age_test', 'type',
       'years_to_test_from_adenoma', 'possible_interruption', 'screening_dxed',
       'false_negative_prob', 'test_result', 'final_dx', 'age_at_final_dx',
       'first_cancerous_time'])

In [47]:
sidebyside.columns = ['age', 'ID', 'gender', 'death', 'adenoma', 'location', 'progressive',
       'AGT', 'SI', 'SII', 'SIII', 'SIV', 'PCT', 'dxed_no_screening',
       'age_at_dx_no_screening', 'age_test', 'type',
       'years_to_test_from_adenoma', 'possible_interruption', 'screening_dxed',
       'false_negative_prob', 'test_result', 'final_dx', 'age_at_final_dx',
       'first_cancerous_time', 'survival_time_not_screened', 'actual_death_not_screened',
       'survival_time_screened', 'actual_death_screened']

In [48]:
sidebyside['death_date_difference'] = sidebyside['actual_death_screened'] - sidebyside['actual_death_not_screened']

In [49]:
sidebyside.head()

#female  == 1 male ==0 for gender 

age    ID  gender       death adenoma location progressive       AGT  \
0   63   326       1   91.289459    True    colon        True  5.981044   
1   73   878       1  100.747850    True    colon        True  1.410019   
2   66  1793       0   81.630761    True   rectum        True  7.664940   
3   70  1793       0   81.630761    True   rectum        True  5.065323   
4   65  2034       0   89.649222    True    colon        True  8.867504   

         SI       SII      SIII       SIV       PCT dxed_no_screening  \
0  0.396665  0.189388  2.791951  0.280256  4.062896               SIV   
1  2.223541  0.425170  0.186668  1.327467  3.854825               SIV   
2  1.341764  0.237118  1.024294  0.709252  2.947965               SIV   
3  0.671101  0.001675  1.436631  1.078467  4.213508               SIV   
4  1.272514  0.447215  1.781689  0.474361  4.915267               SIV   

   age_at_dx_no_screening  age_test   type  years_to_test_from_adenoma  \
0               73.043940      69.0    fit                         6.0   
1               78.264844      75.0  colon                         2.0   
2               76.612905      74.0  colon                         8.0   
3               79.278831      74.0  colon                         4.0   
4               78.782770      66.0  colon                         1.0   

  possible_interruption screening_dxed  false_negative_prob test_result  \
0                  True             SI                 0.30        True   
1                  True             SI                 0.05        True   
2                  True             SI                 0.05        True   
3                  True        Adenoma                 0.15        True   
4                  True        Adenoma                 0.15        True   

  final_dx  age_at_final_dx  first_cancerous_time  survival_time_not_screened  \
0       SI             69.0             68.981044                    0.690906   
1       SI             75.0             74.410019                    0.117777   
2       SI             74.0             73.664940                    3.385859   
3  Adenoma             74.0             75.065323                    1.560722   
4  Adenoma             66.0             73.867504                    1.660622   

   actual_death_not_screened  survival_time_screened  actual_death_screened  \
0                  73.734846              147.404845              91.289459   
1                  78.382621              221.181999             100.747850   
2                  79.998764              101.625314              81.630761   
3                  80.839552              100.000000              81.630761   
4                  80.443393              100.000000              89.649222   

   death_date_difference  
0              17.554613  
1              22.365229  
2               1.631997  
3               0.791208  
4               9.205829

In [50]:
names = pd.read_csv('Ids.csv')

In [51]:
names.head()

Unnamed: 0    ID gender   Names
0           0     6      F    Alex
1           1   655      F  Andrea
2           3  1442      F   Avery
3           6  1968      F   Darcy
4           8  4136      F      Jo

In [52]:
sidebyside.loc[sidebyside.gender ==1, 'gender'] = 'F'
sidebyside.loc[sidebyside.gender ==0, 'gender'] = 'M'

In [53]:
pd.merge(sidebyside, names[['ID', 'gender', 'Names']], right_on=['gender', 'ID'], left_on=['gender', 'ID']).to_csv('15 Side by Side with names.csv')

In [54]:
pd


<module 'pandas' from 'C:\\Users\\M543015\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\pandas\\__init__.py'>

In [55]:
sidebyside

age     ID gender       death adenoma location progressive        AGT  \
0     63    326      F   91.289459    True    colon        True   5.981044   
1     73    878      F  100.747850    True    colon        True   1.410019   
2     66   1793      M   81.630761    True   rectum        True   7.664940   
3     70   1793      M   81.630761    True   rectum        True   5.065323   
4     65   2034      M   89.649222    True    colon        True   8.867504   
5     59   2987      F   78.098218    True   rectum        True   7.127102   
6     51   4148      F   86.733251    True   rectum        True  34.385319   
7     62   4836      F   76.555755    True    colon        True   5.485815   
8     68   5878      F   88.774741    True    colon        True  18.587997   
9     64   6213      M   88.539104    True    colon        True   5.532633   
10    63   6522      M   73.053468    True    colon        True   6.888898   
11    74   6935      F   85.382487    True    colon        True   1.951676   
12    67   7382      F   91.185886    True    colon        True  15.865534   
13    51   7597      M   78.501515    True   rectum        True  17.013961   
14    65   7778      F  103.272314    True   rectum        True   4.703739   
15    53   9566      M   91.077609    True    colon        True  16.949858   
16    70   9796      M   98.775291    True   rectum        True  17.495387   
17    63   9957      M   86.496850    True    colon        True   7.870069   
18    70  10044      F   72.000991    True    colon        True   0.187235   
19    65  10123      F   90.895191    True   rectum        True   3.531241   
20    57  10701      F   99.104290    True    colon        True   9.253511   
21    70  11590      M   93.090509    True   rectum        True   8.554582   
22    75  11648      F   94.248465    True   rectum        True   4.964167   
23    71  12117      M   81.061012    True    colon        True   5.071009   
24    65  13333      M   76.537918    True    colon        True   5.857039   
25    66  13400      M   91.848984    True   rectum        True  14.314932   
26    72  13683      F   94.558124    True    colon        True   7.072340   
27    61  13987      M   94.653106    True    colon        True   6.643157   
28    60  13994      F   91.259062    True   rectum        True   0.700015   
29    66  14021      F   88.804555    True    colon        True   5.472512   
..   ...    ...    ...         ...     ...      ...         ...        ...   
358   78  84239      M   94.270744    True   rectum        True   1.559515   
359   77  84837      M   83.412907    True    colon        True   5.070111   
360   83  85610      M   95.278197    True    colon        True   3.738070   
361   75  85719      F   93.164631    True    colon        True   3.203676   
362   75  86200      F   94.697988    True    colon        True   2.706475   
363   86  86267      M   92.882058    True   rectum        True   2.811629   
364   77  86817      F   87.910508    True   rectum        True   7.993983   
365   78  86872      F   88.289504    True    colon        True   0.354390   
366   87  87991      F   98.944971    True    colon        True   9.108854   
367   78  88247      M   84.411606    True    colon        True   1.043840   
368   75  88287      M   94.599345    True    colon        True  19.238816   
369   77  88925      M   98.333199    True   rectum        True   4.283457   
370   70  89505      M   93.999911    True    colon        True  14.765417   
371   82  90435      M   92.267149    True    colon        True   8.171930   
372   78  90482      M   86.016856    True    colon        True   0.188647   
373   79  90666      M   90.778559    True   rectum        True   7.981266   
374   81  90727      M   92.897130    True   rectum        True   8.651245   
375   84  91032      F   96.599096    True    colon        True   6.024442   
376   81  91202      F   96.617568    True   rectum        True   3.021311   
377   86  93343      M   88